# Fetch OpenDSS Data for SFO Synthetic Grid from Amazon S3 Bucket

### Set up an Amazon AWS Session

In [38]:
# install boto3 into pd environemnt with miniconda using `conda install conda-forge::boto3`
# https://boto3.amazonaws.com/v1/documentation/api/latest/guide/quickstart.html
import boto3
import os # for downloading files

In [13]:
# create an aws account with school email, get access key and secret key
# install aws cli and configure with keys with command `aws configure`
# https://docs.aws.amazon.com/cli/latest/userguide/getting-started-install.html
# https://boto3.amazonaws.com/v1/documentation/api/latest/guide/s3-example-creating-buckets.html
# connect to a specific bucket
s3 = boto3.resource('s3')
oedi = s3.Bucket('oedi-data-lake')

### Explore the Contents of the S3 Bucket

In [47]:
# list and return all objects within a subdirectory
def list_objects(bucket_name, prefix='', num_results=10):
    s3 = boto3.client('s3')
    paginator = s3.get_paginator('list_objects_v2')
    operation_parameters = {'Bucket': bucket_name, 'Prefix': prefix}

    page_iterator = paginator.paginate(**operation_parameters)

    # create an empty list to store the names of the subdirectories
    subdirs = []
    
    for page in page_iterator:
        if 'Contents' in page:
            for obj in page['Contents']:
                subdirs.append(obj['Key'])
                num_results -= 1
                if num_results == 0:
                    return subdirs
    return subdirs

# list and return subdirectories at a specific level
def list_subdirectories_at_level(bucket_name, prefix='', level=1, num_results=10):
    s3 = boto3.client('s3')
    paginator = s3.get_paginator('list_objects_v2')

    # Add a slash to the prefix if it doesn't end with one
    if not prefix.endswith('/'):
        prefix += '/'

    # Calculate the delimiter to use based on the level
    delimiter = '/' * level

    operation_parameters = {'Bucket': bucket_name, 'Prefix': prefix, 'Delimiter': delimiter}

    page_iterator = paginator.paginate(**operation_parameters)

    # create an empty list to store the names of the subdirectories
    subdirs = []

    for page in page_iterator:
        if 'CommonPrefixes' in page:
            for subdir in page['CommonPrefixes']:
                subdirs.append(subdir['Prefix'])
                num_results -= 1
                if num_results == 0:
                    return subdirs
    
    return subdirs
                

In [54]:

# Replace 'your-bucket-name' with the name of your S3 bucket
bucket_name = 'oedi-data-lake'

# Enter the prefix (subdirectory) you want to explore, leave empty for the root directory
prefix_to_explore = 'SMART-DS/v1.0/2018/SFO/'

# list_objects(bucket_name, prefix_to_explore, 10)
subdirs = list_subdirectories_at_level(bucket_name, prefix_to_explore, 1, -1)
print(subdirs)

['SMART-DS/v1.0/2018/SFO/P10U/', 'SMART-DS/v1.0/2018/SFO/P11U/', 'SMART-DS/v1.0/2018/SFO/P12U/', 'SMART-DS/v1.0/2018/SFO/P13U/', 'SMART-DS/v1.0/2018/SFO/P14U/', 'SMART-DS/v1.0/2018/SFO/P15U/', 'SMART-DS/v1.0/2018/SFO/P16U/', 'SMART-DS/v1.0/2018/SFO/P17U/', 'SMART-DS/v1.0/2018/SFO/P18U/', 'SMART-DS/v1.0/2018/SFO/P19U/', 'SMART-DS/v1.0/2018/SFO/P1R/', 'SMART-DS/v1.0/2018/SFO/P1U/', 'SMART-DS/v1.0/2018/SFO/P20U/', 'SMART-DS/v1.0/2018/SFO/P21U/', 'SMART-DS/v1.0/2018/SFO/P22U/', 'SMART-DS/v1.0/2018/SFO/P23U/', 'SMART-DS/v1.0/2018/SFO/P24U/', 'SMART-DS/v1.0/2018/SFO/P25U/', 'SMART-DS/v1.0/2018/SFO/P26U/', 'SMART-DS/v1.0/2018/SFO/P27U/', 'SMART-DS/v1.0/2018/SFO/P28U/', 'SMART-DS/v1.0/2018/SFO/P29U/', 'SMART-DS/v1.0/2018/SFO/P2R/', 'SMART-DS/v1.0/2018/SFO/P2U/', 'SMART-DS/v1.0/2018/SFO/P30U/', 'SMART-DS/v1.0/2018/SFO/P31U/', 'SMART-DS/v1.0/2018/SFO/P32U/', 'SMART-DS/v1.0/2018/SFO/P33U/', 'SMART-DS/v1.0/2018/SFO/P34U/', 'SMART-DS/v1.0/2018/SFO/P35U/', 'SMART-DS/v1.0/2018/SFO/P3R/', 'SMART-DS/v1

### Download Files within a Subdirectory

In [41]:
def download_files(bucket_name, prefix, local_directory, num_files=-1):
    s3 = boto3.client('s3')
    paginator = s3.get_paginator('list_objects_v2')
    operation_parameters = {'Bucket': bucket_name, 'Prefix': prefix}

    page_iterator = paginator.paginate(**operation_parameters)

    for page in page_iterator:
        if 'Contents' in page:
            for obj in page['Contents']:
                key = obj['Key']
                if key.endswith('/'):
                    continue  # Skip directories

                # Construct local file path
                local_file_path = os.path.join(local_directory, os.path.basename(key))

                # Create local directory if it doesn't exist
                os.makedirs(os.path.dirname(local_file_path), exist_ok=True)

                # Download file from S3
                s3.download_file(bucket_name, key, local_file_path)
                print(f'Downloaded: {key} to {local_file_path}')

                # Decrement the number of files to download
                num_files -= 1
                if num_files == 0:
                    return


In [60]:

# Replace 'your-bucket-name' with the name of your S3 bucket
bucket_name = 'oedi-data-lake'

# which data to download
project = 'SMART-DS'
version = 'v1.0'
year = '2018'
location = 'SFO'
region = 'P13U'
data_type = 'scenarios'
scenario = 'base_timeseries'
data_format = 'geojson'

sub_dir = f'{data_type}/{scenario}/{data_format}/'

# Specify the local directory where you want to save the downloaded files
local_path_prefix = '/Users/ilyonsg/Documents/nrel/data/sfo/'


In [61]:
# loop over the regions to download the files
i = 0
n = len(subdirs)
for prefix in subdirs:
    i += 1
    object_prefix = f'{prefix}{sub_dir}'
    # get the region as the last part of the prefix
    region = prefix.split('/')[-2]
    print("Downloading region", i, "of", n, region, "...")
    local_path = f'{local_path_prefix}/{scenario}/{region}/'
    download_files(bucket_name, object_prefix, local_path, 0)

Downloaded: SMART-DS/v1.0/2018/SFO/P10U/scenarios/base_timeseries/geojson/p10uhs0_1247--p10udt2190.json to /Users/ilyonsg/Documents/nrel/data/sfo//base_timeseries/P10U/p10uhs0_1247--p10udt2190.json
Downloaded: SMART-DS/v1.0/2018/SFO/P10U/scenarios/base_timeseries/geojson/p10uhs0_1247--p10udt2438.json to /Users/ilyonsg/Documents/nrel/data/sfo//base_timeseries/P10U/p10uhs0_1247--p10udt2438.json
Downloaded: SMART-DS/v1.0/2018/SFO/P10U/scenarios/base_timeseries/geojson/p10uhs0_1247--p10udt2499.json to /Users/ilyonsg/Documents/nrel/data/sfo//base_timeseries/P10U/p10uhs0_1247--p10udt2499.json
Downloaded: SMART-DS/v1.0/2018/SFO/P10U/scenarios/base_timeseries/geojson/p10uhs0_1247--p10udt2508.json to /Users/ilyonsg/Documents/nrel/data/sfo//base_timeseries/P10U/p10uhs0_1247--p10udt2508.json
Downloaded: SMART-DS/v1.0/2018/SFO/P10U/scenarios/base_timeseries/geojson/p10uhs1_1247--p10udt189.json to /Users/ilyonsg/Documents/nrel/data/sfo//base_timeseries/P10U/p10uhs1_1247--p10udt189.json
Downloaded: 